In [ ]:
import numpy as np
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML
from quick_sim import setup_sim
from edward_tools.coupled_fq_potential import coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, coupled_flux_qubit_non_linear_approx_pot_break_down
import edward_tools.fq_runner as fq_runner
from edward_tools.visualization import animate_sim_flux_qubit
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
from PARAMETER_INPUT import *
from PARAMETER_INPUT import _lambda, _theta, _eta
from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time

import kyle_tools as kt
import matplotlib.pyplot as plt

import importlib, os, datetime
from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library
import edward_tools.cfq_batch_sweep as cfq_batch_sweep



coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

In [ ]:
PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 0.5
k_BT = k_B * T

In [ ]:

C_factor = 100
L_factor = 0.35
I_m_factor = 15
I_p_1, I_p_2 = 2e-6 , 2e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                                # Amp
C_1, C_2 = 4e-9 * C_factor, 4e-9 * C_factor                              # F
L_1, L_2 = 1e-9 * L_factor, 1e-9 * L_factor                             # H 

R_1, R_2 = 371, 371                                # ohm

quick_doubler = lambda x1, x2: np.hstack([np.array([x1] * 2), np.array([x2]*2)])
I_p, I_m = quick_doubler(I_p_1, I_p_2), quick_doubler(I_m_1, I_m_2)

m_c = C_1
m_1 = C_1
m_2 = C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)


U0_1 = m_c * x_c**2 / t_c**2
U0_2 = m_2 * x_c**2 / t_c**2
kappa_1, kappa_2, kappa_3, kappa_4 = k_BT/U0_1, k_BT/U0_1, k_BT/U0_1, k_BT/U0_1 # these are kappa' in the equation

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

gamma = 20


beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;

d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;


_lambda = np.array([lambda_1, lambda_2, lambda_3, lambda_4])
_theta  = np.array([theta_1, theta_2, theta_3, theta_4])
_eta  =   np.array([eta_1, eta_2, eta_3, eta_4])

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 1000
params['dt'] = 1/100
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['comment'] = "test the effect of I_m asymmetry"



In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor

coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, 14, 4,\
                           default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
# phi_1_dcx, phi_2_dcx = 3, 3
phi_1_dcx, phi_2_dcx = 0, 0
phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx

phi_1_dcx_off = 0
phi_2_dcx_off = 0
M_12_off = 0

# bookmark
# phi_1_dcx_on = 2.8
# phi_2_dcx_on = 2.8
# M_12_on = -0.6

phi_1_dcx_on = 3.0
phi_2_dcx_on = 3.0
M_12_on = -0.6

initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": 0,  "phi_2_x": 0,  "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_1_dcx_off,
        "M_12": M_12_off, 'x_c': x_c
}

In [ ]:
zeroDissipation = False

params['sim_params'] = [_lambda, _theta, _eta]
if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

TR_initial_condition = [
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off), 
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_off), 
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_on), 
    (phi_1_dcx_off, phi_2_dcx_off, M_12_on), 
    (phi_1_dcx_on,  phi_2_dcx_off, M_12_off),
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off) 
]

protocol_index = 4

# bookmark
# initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = TR_initial_condition[protocol_index-1]
initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = \
TR_initial_condition[protocol_index]

rest = lambda t: {"duration": t, "name":"rest"}
duration_t = 200
protocol_list = [
    # forward
    # {"duration":50, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on * 0.5,  "M_12": M_12_off,  "name":"(1) mix in y direction"},
    # {"duration":200, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_off,  "name":"(1) mix in y direction"},
    # {"duration":60, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_on,   "name":"(2) conditional tilt"},
    # # {"duration":100, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_on,   "name":"(3) raise the barrier"},
    # {"duration":100, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_on,  "name": "(4) conditional tilt in x"}, 
    # {"duration":60, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(5) mix in x direction"}, 
    {"duration":200, "phi_1_dcx": phi_1_dcx_on * 0.5, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential"}, 
    {"duration":50, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential"}, 
]


protocol_time_array = [item["duration"] for item in protocol_list]
protocol_time_array.insert(0, 0)
protocol_time_array = np.cumsum(protocol_time_array)

In [ ]:

"""
# step 3: create the relevant storage protocol and computation protocol
"""
computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict)

In [ ]:
"""
# step 4: create the coupled_fq_runner
"""
cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol, measure_all_states=True)
cfqr.initialize_sim()
# cfqr.set_sim_attributes(extra_constraint=extra_constraint_00_and_11_only)
cfqr.set_sim_attributes()
init_state_saved = cfqr.init_state

In [ ]:
N_particle, d1, d2 = cfqr.init_state.shape

In [ ]:
combined_init_state = np.empty([N_particle * 2, d1, d2])
label_column = np.empty([N_particle * 2], dtype = object)

In [ ]:
combined_init_state[:N_particle, ...] = cfqr.init_state
label_column[:1000][init_state_saved[:1000, 1, 0]>0] = "10"
label_column[:1000][init_state_saved[:1000, 1, 0]<0] = "00"

In [ ]:
combined_init_state[N_particle:, ...] = cfqr.init_state
label_column[1000:][init_state_saved[:1000, 1, 0]>0] = "11"
label_column[1000:][init_state_saved[:1000, 1, 0]<0] = "01"

In [ ]:
x_coord_00 = combined_init_state[label_column=="00"][:, 0, 0]
y_coord_00 = combined_init_state[label_column=="00"][:, 1, 0]
x_coord_01 = combined_init_state[label_column=="01"][:, 0, 0]
y_coord_01 = combined_init_state[label_column=="01"][:, 1, 0]
x_coord_10 = combined_init_state[label_column=="10"][:, 0, 0]
y_coord_10 = combined_init_state[label_column=="10"][:, 1, 0]
x_coord_11 = combined_init_state[label_column=="11"][:, 0, 0]
y_coord_11 = combined_init_state[label_column=="11"][:, 1, 0]

In [ ]:
plt.xlim([-4, 4])
plt.ylim([-4, 4])
plt.scatter(x_coord_00, y_coord_00, c = "blue")
plt.scatter(x_coord_10, y_coord_10, c = "green")
plt.scatter(x_coord_01, y_coord_01, c = "orange")
plt.scatter(x_coord_11, y_coord_11, c = "red")


In [ ]:
def graphForCriticalValueOfPhi_xdc(parameterArray):
    XYU_array = []
    
    final_t = 40
    for p_array in parameterArray:   
        print(p_array)
        protocol_list = [
            {"duration":final_t, "phi_1_dcx": p_array[0], "phi_2_dcx": p_array[1],  "M_12": p_array[2],  "name":"(1) mix in y direction"},
        ]
        computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, protocol_list)
        storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict)
        cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, storage_protocol= storage_protocol, computation_protocol= comp_protocol)
        cfqr.initialize_sim(); 
        cfqr.set_sim_attributes()
        init_state_saved = cfqr.init_state
        
        
        fig, ax = plt.subplots(1, 2, figsize = [12, 6])
        X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = final_t, cutlineDirection = "v")
        plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = -1.96436202,\
                                    contour_plt=ax[0], cutline_plt=ax[1], contours = 20,
                                    showGraph=True)

        XYU_array.insert(0, plt_line_data_at_t)
        
    return XYU_array

In [ ]:
# XYU_array = graphForCriticalValueOfPhi_xdc([(0, 2.6,0), (0, 2.8, 0), (0, 3.0, 0)])

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize = [14, 6])
# for index, item in enumerate(XYU_array):

#     X, Y, U = item["contour_plot"]["X"], item["contour_plot"]["Y"], item["contour_plot"]["U"]
#     contours, time = item["contour_plot"]["contours"], item["contour_plot"]["time"]
#     _plotAxis = item["cutline_plot"]["plotAxis"]
#     _targetU = item["cutline_plot"]["targetU"]
#     ax[0].set_box_aspect(1)
#     ax[1].set_box_aspect(1)

#     ax[0].set_xlabel(r"$\varphi_{1}$", fontsize = 20)
#     ax[0].set_ylabel(r"$\varphi_{2}$", fontsize = 20)
#     ax[0].xaxis.set_tick_params(labelsize=15)
#     ax[0].yaxis.set_tick_params(labelsize=15)

    
#     ax[1].set_ylabel("potential energy", fontsize = 18)
#     ax[1].set_xlabel(r"$\varphi_{2}$", fontsize = 20)
#     ax[1].xaxis.set_tick_params(labelsize=15)
#     ax[1].yaxis.set_tick_params(labelsize=15)

    
#     ax[0].contourf(X, Y, U, contours)
#     ax[0].vlines(x = -2.6, ymin = -5, ymax = 5, colors= "red")
#     ax[1].plot(_plotAxis, _targetU, label = r"$\varphi_{2dcx}$ = " + f"{phi_1_dcx_on_array[index]}")
#     # ax[1].legend(fontsize= 15)
# #     ax[1].set_title(, fontsize = 15)
# # "contour_plot": {"X": X, "Y": Y, "U": U, "contours": contours, "time": time},
# #         "cutline_plot": {"plotAxis": plotAxis, "targetU": targetU, "time": time, "cutlineDirection": cutlineDirection, "cutlineValue": cutlineValue}

# actual simulations

In [ ]:
manual_domain=[np.array([-5, -5])/time_scale_factor, np.array([5, 5])/time_scale_factor]
mapping_state_1_to_state_2_dict = {"00": ["00", "10"], "01": ["00", "10"], "10": ["01", "11"], "11": ["01", "11"]}

In [ ]:
# step 5: perform simulations

simResult = cfq_batch_sweep.simulateSingleCoupledFluxQubit(params, initial_parameter_dict, protocol_list, \
                        potential = coupled_fq_pot, potential_default_param = coupled_fq_default_param,            
                        initial_state = combined_init_state, manual_domain = manual_domain, \
                        phi_1_dcx = phi_1_dcx,  phi_2_dcx = phi_2_dcx, \
                        percentage = 1, as_step = np.s_[::], measure_all_states=True)

In [ ]:
# animations
vmin, vmax = 0, 100
frame_skip = 1000
# phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx
particle_number = 0
# all_state = simResult["cfqr"].sim.output.all_state['states'][particle_number:particle_number+1, :, :, :]
all_state = simResult["cfqr"].sim.output.all_state['states']
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(all_state, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

In [ ]:
# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
final_state = simResult['cfqr'].sim.output.final_state[:,(0, 1), 0]

In [ ]:
final_state_in_00 = np.logical_and(final_state[:, 0] < 0, final_state[:, 1] < 0)
final_state_in_01 = np.logical_and(final_state[:, 0] < 0, final_state[:, 1] > 0)
final_state_in_10 = np.logical_and(final_state[:, 0] > 0, final_state[:, 1] < 0)
final_state_in_11 = np.logical_and(final_state[:, 0] > 0, final_state[:, 1] > 0)

In [ ]:
ratio_00_left = np.sum(np.logical_and(final_state_in_00, label_column == "00")) / np.sum(label_column == "00")
ratio_00_right = np.sum(np.logical_and(final_state_in_10, label_column == "00")) / np.sum(label_column == "00")
print(f"ratio for 00 = {ratio_00_left:.3g}:{ratio_00_right:.3g}")

In [ ]:
ratio_01_left = np.sum(np.logical_and(final_state_in_00, label_column == "01")) / np.sum(label_column == "00")
ratio_01_right = np.sum(np.logical_and(final_state_in_10, label_column == "01")) / np.sum(label_column == "00")
print(f"ratio for 01 = {ratio_01_left:.3g}:{ratio_01_right:.3g}")

In [ ]:
ratio_10_left = np.sum(np.logical_and(final_state_in_01, label_column == "10")) / np.sum(label_column == "00")
ratio_10_right = np.sum(np.logical_and(final_state_in_11, label_column == "10")) / np.sum(label_column == "00")
print(f"ratio for 01 = {ratio_10_left:.3g}:{ratio_10_right:.3g}")

In [ ]:
ratio_11_left = np.sum(np.logical_and(final_state_in_01, label_column == "11")) / np.sum(label_column == "00")
ratio_11_right = np.sum(np.logical_and(final_state_in_11, label_column == "11")) / np.sum(label_column == "00")
print(f"ratio for 01 = {ratio_11_left:.3g}:{ratio_11_right:.3g}")

In [ ]:
# %%notify
step_time_array = np.cumsum(np.array([x["duration"] for x in protocol_list]))/params['dt']
name_array = [x["name"] for x in protocol_list]


plt.hist(simResult["work_distribution"], bins = 45)
plt.show()
mean_work = np.mean(simResult["work_distribution"])
jarzyn_term = np.mean(np.exp(-simResult["work_distribution"]))
print("sim_id = ", simResult['simulation_data'])
print(f"N = {params['N']}, dt = {params['dt']}")
print(f"phi_1dcx_on: {phi_1_dcx_on}, phi_2dcx_on: {phi_2_dcx_on}, M_12_on: {M_12_on}")
print(f"L_factor = {L_factor}, C_factor = {C_factor}, I_m_factor = {I_m_factor}, gamma = {gamma}")
print([x["duration"] for x in protocol_list])
print(f'jarzyn = {jarzyn_term}, mean work = {mean_work}')


In [ ]:

step_array = np.arange(simResult["work_statistic"][:,0].shape[0])
skip_step = int(len(step_array) * 0.01)
work_mean = simResult["work_statistic"][:,0]
work_std = simResult["work_statistic"][:,1]

coarse_step_array = step_array[::skip_step]
coarse_step_array = np.append(coarse_step_array, step_array[-1])
coarse_work_mean_array = work_mean[::skip_step]
coarse_work_mean_array = np.append(coarse_work_mean_array, work_mean[-1])
coarse_work_std_array = np.append(work_std[::skip_step], work_std[-1])

plt.errorbar(coarse_step_array, coarse_work_mean_array, yerr = coarse_work_std_array)
# plt.yscale('log')

for i, t in enumerate(step_time_array):
    plt.axvline(x = t, color = 'b', label = 'axvline - full height')
    # plt.text('2017-07-02 16:30:00',0.005,'BigNews1',rotation=90,va='top')
    plt.annotate(f"{i+1}", 
            xy=(t, 4.5),  
            # xytext=(quantile_75, 2.05*1e9),
            color='red', 
            ha='center', 
            size=16)

first_half = work_mean[0: len(work_mean)//2]
second_half = work_mean[len(work_mean)//2:][::-1]
abs_error = np.abs(first_half - second_half)
# plt.plot(step_array[0:len(step_array)//2], abs_error)
print(max(abs_error), min(abs_error))
print(f'jarzyn = {jarzyn_term}')
print([x["duration"] for x in protocol_list])

In [ ]:
final_state_in_00

In [ ]:
simResult["fidelity"]

In [ ]:
import importlib
cfq_batch_sweep = importlib.reload(cfq_batch_sweep)

In [ ]:
cfq_batch_sweep.saveSimulationResult(simResult, U0_1, timeOrStep = 'step', save = True, save_final_state = False, saveFolderPath = "coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery", save_all_state = False, save_work_distribution_time_array = True)

# minimum value of potential energy

In [ ]:
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
import importlib
importlib.reload(minimum_value_of_potential)

In [ ]:
potential_type_and_time_array = list(zip(protocol_time_array, ["four_well", "mix_in_y", "mix_in_y", "conditional_tilt_in_y", "conditional_tilt_in_x", "mix_in_x", "mix_in_x", "four_well"]))
minimum_point_data_array = minimum_value_of_potential.obtain_minimum_points(cfqr, potential_type_and_time_array, beta_1, beta_2, d_beta_1, d_beta_2, verbose = False)
offset_value = np.array([d['min_E'] for d in minimum_point_data_array])
minimum_points_location = np.array([d['coordinate'] for d in minimum_point_data_array])
minimum_points_potential = np.array([d['min_E'] for d in minimum_point_data_array])
offset_energy = minimum_points_potential[1:] - minimum_points_potential[:-1]
energy_profile_array = []
contour_value = 40
# print(minimum_points, minimum_points_potential)
for x in minimum_point_data_array:
    print(x)

In [ ]:
from scipy.optimize import fmin

In [ ]:
def find_minimum_for_all_potential_2(cfqr, _t, beta_1, beta_2, d_beta_1, d_beta_2, verbose = True, guess = [(0, 0)]):
    _params_at_t = cfqr.system.protocol.get_params(_t)
    _phi_1x = 0
    _phi_2x = 0
    _phi_1dcx = _params_at_t[10]
    _phi_2dcx = _params_at_t[11]
    _phi_1dc = _phi_1dcx
    _phi_2dc = _phi_2dcx
    _M_12 = _params_at_t[12]
    _xi = 1 / (1 - _M_12**2)
    
    def Fcn(coord):
        _phi_1, _phi_2 = coord
        u1_1 = 1/2 * _xi * (_phi_1 - _phi_1x)**2
        u3_1 = beta_1 * np.cos(_phi_1) * np.cos(_phi_1dc/2)
        u4_1 = d_beta_1 * np.sin(_phi_1) * np.sin(_phi_1dc/2)
        
        u1_2 = 1/2 * _xi * (_phi_2 - _phi_2x)**2        
        u3_2 = beta_2 * np.cos(_phi_2) * np.cos(_phi_2dc/2)
        u4_2 = d_beta_2 * np.sin(_phi_2) * np.sin(_phi_2dc/2)
        u5 = _M_12 * _xi * (_phi_1 - _phi_1x) * (_phi_2 - _phi_2x)
        
        return u1_1 + u1_2 + u3_1 + u3_2 + u4_1 + u4_2 + u5

    solution_set = [fmin(Fcn, _g, disp=False) for _g in guess]
    sol_1 = list(solution_set[0])
    _potential_break_down = coupled_flux_qubit_non_linear_approx_pot_break_down(sol_1[0], sol_1[1], _phi_1dcx, _phi_2dcx, _params_at_t)
    # _min_potential = cfqr.system.potential.potential(sol_1[0], sol_1[1], _phi_1dcx, _phi_2dcx, _params_at_t)
    return sol_1, _potential_break_down

In [ ]:
protocol_time_array

## 4 well potential

In [ ]:
_t_array = np.array(simResult['cfqr'].sim.output.all_state["step_indices"]) * params['dt']

In [ ]:
min_point_sol_guess = [[-2, -1.2]]
min_point_set = []
skip_step = 100
for _i, _t in enumerate(_t_array[::skip_step]):
    _min_point_set = find_minimum_for_all_potential_2(cfqr, _t, beta_1, beta_2, d_beta_1, d_beta_2, verbose = False, 
                                 guess = [min_point_sol_guess[0]])
    min_point_sol_guess.append(_min_point_set[0])
    min_point_set.append(_min_point_set)
    

In [ ]:
min_point_coord = np.array([d[0] for d in min_point_set])
min_point_energy_break_down = np.array([list(d[1].values()) for d in min_point_set])

In [ ]:
protocol_time_array

In [ ]:
plt.plot(_t_array[::skip_step], min_point_energy_break_down[:, 3], label = "u4_1") # u4_1 = delta_beta_1 * np.sin(phi_1) * np.sin(phi_1dc/2)
plt.plot(_t_array[::skip_step], min_point_energy_break_down[:, 7], label = "u4_2") # u4_2 = delta_beta_2 * np.sin(phi_2) * np.sin(phi_2dc/2)
[plt.vlines(x=_t, ymin=-1, ymax = 1, linestyles="dashed") for _t in protocol_time_array]
plt.legend()
plt.show()


In [ ]:
plt.plot(_t_array[::skip_step], min_point_energy_break_down[:, 0], label = "u1_1") # u1_1 = 1/2 * (phi_1 - phi_1x)**2 * g_1
plt.plot(_t_array[::skip_step], min_point_energy_break_down[:, 1], label = "u2_1") # u2_1 = 1/2 * g_1 * (phi_1dc - phi_1dcx)**2
plt.plot(_t_array[::skip_step], min_point_energy_break_down[:, 2], label = "u3_1") # u3_1 = beta_1 * np.cos(phi_1) * np.cos(phi_1dc/2)
plt.plot(_t_array[::skip_step], min_point_energy_break_down[:, 3], label = "u4_1") # u4_1 = delta_beta_1 * np.sin(phi_1) * np.sin(phi_1dc/2)
[plt.vlines(x=_t, ymin=-60, ymax = 200, linestyles="dashed") for _t in protocol_time_array]
plt.legend()
plt.show()


plt.plot(_t_array[::skip_step], min_point_energy_break_down[:, 4], label = "u1_2") # u1_2 = 1/2 * (phi_2 - phi_2x)**2
plt.plot(_t_array[::skip_step], min_point_energy_break_down[:, 5], label = "u2_2") # u2_2 = 1/2 * g_2 * (phi_2dc - phi_2dcx)**2
plt.plot(_t_array[::skip_step], min_point_energy_break_down[:, 6], label = "u3_2") # u3_2 = beta_2 * np.cos(phi_2) * np.cos(phi_2dc/2)
plt.plot(_t_array[::skip_step], min_point_energy_break_down[:, 7], label = "u4_2") # u4_2 = delta_beta_2 * np.sin(phi_2) * np.sin(phi_2dc/2)
[plt.vlines(x=_t, ymin=-60, ymax = 200, linestyles="dashed") for _t in protocol_time_array]
plt.legend()
plt.show()

plt.plot(_t_array[::skip_step], min_point_energy_break_down[:, 8], label = "u5")   # u5 = M_12 * (phi_1 - phi_1x) * (phi_2 - phi_2x)
[plt.vlines(x=_t, ymin=-200, ymax = 200, linestyles="dashed") for _t in protocol_time_array]
plt.legend()

In [ ]:
plt.scatter(min_point_coord[:, 0], min_point_coord[:, 1])

In [ ]:
np.save("coupled_flux_qubit_protocol/fixed_points.npy", min_point_set, )

In [ ]:
def find_minimum_for_all_asym_potential_2(cfqr, _t, beta_1, beta_2, d_beta_1, d_beta_2, verbose = True, guess = [(0, 0)]):
    _params_at_t = cfqr.system.protocol.get_params(_t)
    _phi_1x = 0
    _phi_2x = 0
    _phi_1dcx = _params_at_t[10]
    _phi_2dcx = _params_at_t[11]
    _phi_1dc = _phi_1dcx
    _phi_2dc = _phi_2dcx
    _M_12 = _params_at_t[12]
    _xi = 1 / (1 - _M_12**2)
    x_1 = -1.964
    x_2 = -1.556
    def Fcn(coord):
        a, b = coord
        u1 = -_xi * x_1 * a + 1/2 * _xi * a**2 - d_beta_1 * np.sin(x_1) * np.sin(_phi_1dc/2)
        u2 = -_xi * x_2 * b + 1/2 * _xi * b**2 - d_beta_2 * np.sin(x_2) * np.sin(_phi_2dc/2)
        u3 = _M_12 * _xi * a * b - _M_12 * _xi * a * x_2 - _M_12 * _xi * b * x_1
        
        return u1 + u2 + u3

    solution_set = [fmin(Fcn, _g, disp=False) for _g in guess]
    sol_1 = list(solution_set[0])
    # _min_potential = cfqr.system.potential.potential(sol_1[0], sol_1[1], _phi_1dcx, _phi_2dcx, _params_at_t)
    _potential_break_down = coupled_flux_qubit_non_linear_approx_pot_break_down(sol_1[0], sol_1[1], _phi_1dcx, _phi_2dcx, _params_at_t)
    return sol_1, _potential_break_down

In [ ]:
min_point_sol_guess = [[-2, -1.2]]
min_point_set = []
for _i, _t in enumerate(_t_array[::100]):
    _min_point_set = find_minimum_for_all_potential_2(cfqr, _t, beta_1, beta_2, d_beta_1, d_beta_2, verbose = False, 
                                 guess = [min_point_sol_guess[0]])
    min_point_sol_guess.append(_min_point_set[0])
    min_point_set.append(_min_point_set)
    

In [ ]:
min_point_set[0][1].keys()

In [ ]:
energy_break_down = np.array([list(d.values()) for d in min_point_set[:,1]])

In [ ]:
plt.plot(_t_array[::25], energy_break_down[:, 0], label = "u1_1") # u1_1 = 1/2 * (phi_1 - phi_1x)**2 * g_1
plt.plot(_t_array[::25], energy_break_down[:, 2], label = "u2_1") # u2_1 = 1/2 * g_1 * (phi_1dc - phi_1dcx)**2
plt.plot(_t_array[::25], energy_break_down[:, 4], label = "u3_1") # u3_1 = beta_1 * np.cos(phi_1) * np.cos(phi_1dc/2)
plt.plot(_t_array[::25], energy_break_down[:, 6], label = "u4_1") # u4_1 = delta_beta_1 * np.sin(phi_1) * np.sin(phi_1dc/2)
plt.legend()
plt.show()

plt.plot(_t_array[::25], energy_break_down[:, 1], label = "u1_2") # u1_2 = 1/2 * (phi_2 - phi_2x)**2
plt.plot(_t_array[::25], energy_break_down[:, 3], label = "u2_2") # u2_2 = 1/2 * g_2 * (phi_2dc - phi_2dcx)**2
plt.plot(_t_array[::25], energy_break_down[:, 5], label = "u3_2") # u3_2 = beta_2 * np.cos(phi_2) * np.cos(phi_2dc/2)
plt.plot(_t_array[::25], energy_break_down[:, 7], label = "u4_2") # u4_2 = delta_beta_2 * np.sin(phi_2) * np.sin(phi_2dc/2)
plt.legend()
plt.show()

plt.plot(_t_array[::25], energy_break_down[:, 8], label = "u5")   # u5 = M_12 * (phi_1 - phi_1x) * (phi_2 - phi_2x)
plt.legend()

In [ ]:
x_coord = [d[0] for d in min_point_set[:, 0]]
y_coord = [d[1] for d in min_point_set[:, 0]]
plt.scatter(x_coord, y_coord)

In [ ]:
# selected_particles  = simResult["cfqr"].sim.output.all_state["states"][:, :, :, 0]
# _, N_step_number, _ = selected_particles.shape
# step_time_array = np.array(simResult["cfqr"].sim.output.all_state["step_indices"]) * params['dt']

# def findPotential(cfqr, _t, state):
#     _params_at_t = cfqr.system.protocol.get_params(_t)
#     def cfq_potential(state):
#         phi_1, phi_2, phi_1dc, phi_2dc = state
#         return coupled_flux_qubit_non_linear_approx_pot_break_down(phi_1, phi_2, phi_1dc, phi_2dc, _params_at_t)
#     return np.array(list(cfq_potential(state).values()))



# for i in range(1, len(step_time_array))[::1000]:
#     # print(step_time_array[i], selected_particles[i, :])
#     print( selected_particles[:, i-1, :].shape)
#     U_a[i-1] = findPotential(simResult["cfqr"], step_time_array[i], selected_particles[:, i-1, :])
#     # U_a[i-1] = findPotential(simResult["cfqr"], step_time_array[i], selected_particles[i-1, :])
#     U_b[i-1] = findPotential(simResult["cfqr"],  step_time_array[i-1], selected_particles[i-1, :])

In [ ]:
# find the work done of each particle

In [ ]:
potential_array = []
for _index, _t in enumerate(step_time_array):
    _params_at_t = simResult['cfqr'].system.protocol.get_params(_t)
    def cfq_potential(state):
        # print(state)
        phi_1, phi_2, phi_1dc, phi_2dc = state
        return coupled_flux_qubit_non_linear_approx_pot(phi_1, phi_2, phi_1dc, phi_2dc, _params_at_t)
    
    
    vectorized_cfq_potential = np.vectorize(cfq_potential, signature='(4)->()')
    # print(selected_particles[:, _index, :, 0])
    potential_array.append(vectorized_cfq_potential(selected_particles[:, _index, :]))
    
    

In [ ]:
selected_particles  = simResult["cfqr"].sim.output.all_state["states"][0:10,...]
N_particle_number, N_step_number, _, _ = selected_particles.shape
step_time_array = np.array(simResult["cfqr"].sim.output.all_state["step_indices"]) * params['dt']

In [ ]:
break_down_potential_categories = ["u1_1", "u2_1",  "u3_1", "u4_1", "u1_2", "u2_2", "u3_2", "u4_2", "u5"]
U_a = np.zeros([N_particle_number, N_step_number-1]) 
U_b = np.zeros([N_particle_number, N_step_number-1])

In [ ]:
for i in range(1, len(step_time_array)):
    # print(step_time_array[i], selected_particles[i, :])
    U_a[i-1] = findPotential(simResult["cfqr"], step_time_array[i], selected_particles[:, i-1, :, 0])
    # U_a[i-1] = findPotential(simResult["cfqr"], step_time_array[i], selected_particles[i-1, :])
    U_b[i-1] = findPotential(simResult["cfqr"],  step_time_array[i-1], selected_particles[:, i-1, :, 0])

# For n particles

## trial 1

# For one particle 

In [ ]:
selected_particles  = simResult["cfqr"].sim.output.all_state["states"][0, :, :, 0]
N_step_number, _ = selected_particles.shape
step_time_array = np.array(simResult["cfqr"].sim.output.all_state["step_indices"]) * params['dt']

U_a = np.zeros( N_step_number-1) 
U_b = np.zeros(N_step_number-1)


def findPotential(cfqr, _t, state):
    # print(state)
    _params_at_t = cfqr.system.protocol.get_params(_t)
    def cfq_potential(state):
        phi_1, phi_2, phi_1dc, phi_2dc = state
        return coupled_flux_qubit_non_linear_approx_pot(phi_1, phi_2, phi_1dc, phi_2dc, _params_at_t)
    return cfq_potential(state)



for i in range(1, len(step_time_array)):
    # print(selected_particles[i-1, :].shape)
    # U_a[i-1] = findPotential(simResult["cfqr"], step_time_array[i], selected_particles[:, i-1, :])
    U_a[i-1] = findPotential(simResult["cfqr"], step_time_array[i], selected_particles[i-1, :])
    U_b[i-1] = findPotential(simResult["cfqr"],  step_time_array[i-1], selected_particles[i-1, :])

In [ ]:
selected_particles.shape

In [ ]:
np.sum(U_a - U_b)

In [ ]:
len(step_time_array)

In [ ]:
selected_particles  = simResult["cfqr"].sim.output.all_state["states"][0, :, :, 0]
N_step_number, _ = selected_particles.shape
step_time_array = np.array(simResult["cfqr"].sim.output.all_state["step_indices"]) * params['dt']

U_a = np.zeros( N_step_number-1) 
U_b = np.zeros(N_step_number-1)


def findPotential(cfqr, _t, state):
    print(state.shape)
    _params_at_t = cfqr.system.protocol.get_params(_t)
    def cfq_potential(state):
        phi_1, phi_2, phi_1dc, phi_2dc = state
        return coupled_flux_qubit_non_linear_approx_pot(phi_1, phi_2, phi_1dc, phi_2dc, _params_at_t)
    vectorized_cfq_potential = np.vectorize(cfq_potential, signature='(4)->()')
    return cfq_potential(state)




U_a[i-1] = findPotential(simResult["cfqr"], step_time_array[i], selected_particles)
# U_b[i-1] = findPotential(simResult["cfqr"],  step_time_array[i-1], selected_particles[i-1, :])

In [ ]:


def plot_work_dist_with_time(sim_id_1, sim_id_2):
    work_dist_time_array_1 = np.load(f"coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{sim_id_1}__work_distribution_time_array.npy")
    work_dist_time_array_2 = np.load(f"coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery/{sim_id_2}__work_distribution_time_array.npy")
    step_time_array = np.array(cfqr.sim.output.all_state["step_indices"]) * params['dt']
    
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(5,5)
    points = [(0.1, 0.5), (0.5, 0.5), (0.9, 0.5)]
    _skipped_work_dist_time_array = work_dist_time_array
    _skipped_step_time_array = step_time_array[::skipNumber]
    
    def animate(i):
        ax.clear()
        # Get the point from the points list at index i
        _dist = _skipped_work_dist_time_array[i]
        work_min = np.min(_dist) - 5
        work_max = np.max(_dist) + 5
        bins = np.linspace(work_min, work_max, 50)

        # Plot that point using the x and y coordinates
        ax.hist(_dist, bins = bins)
        ax.set_title(f"t = {_skipped_step_time_array[i]}")
        # Set the x and y axis to display a fixed range
    ani = FuncAnimation(fig, animate, frames=len(_skipped_work_dist_time_array),
                        interval=100, repeat=False)
    plt.close()
    return ani

In [ ]:
def plot_work_dist_with_time(cfqr, skipNumber = 100):
    work_dist_time_array = cfqr.sim.work_dist_time_array
    step_time_array = np.array(cfqr.sim.output.all_state["step_indices"]) * params['dt']
    
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(5,5)
    points = [(0.1, 0.5), (0.5, 0.5), (0.9, 0.5)]
    _skipped_work_dist_time_array = work_dist_time_array[::skipNumber]
    _skipped_step_time_array = step_time_array[::skipNumber]
    
    def animate(i):
        ax.clear()
        # Get the point from the points list at index i
        _dist = _skipped_work_dist_time_array[i]
        work_min = np.min(_dist) - 5
        work_max = np.max(_dist) + 5
        bins = np.linspace(work_min, work_max, 50)

        # Plot that point using the x and y coordinates
        ax.hist(_dist, bins = bins)
        ax.set_title(f"t = {_skipped_step_time_array[i]}")
        # Set the x and y axis to display a fixed range
    ani = FuncAnimation(fig, animate, frames=len(_skipped_work_dist_time_array),
                        interval=100, repeat=False)
    plt.close()
    return ani

In [ ]:
work_dist_time_array = simResult['cfqr'].sim.work_dist_time_array

In [ ]:
html = display.HTML(plot_work_dist_with_time(simResult['cfqr']).to_html5_video())
display.display(html)


In [ ]:
import matplotlib.animation as animation

writer = animation.PillowWriter(fps=15,
                                metadata=dict(artist='Me'),
                                bitrate=1800)
ani.save('I_m_0nA.gif', writer=writer)


In [ ]:
from scipy import optimize
import numpy as np

In [ ]:
phi_1 = np.linspace(-3.5, 3.5, 1000)
phi_1x = 0

In [ ]:
def U_1(phi_1, phi_1x, phi_1dc):
    _potential = 1/2 * (phi_1 - phi_1x)**2 + beta_1 * np.cos(phi_1) * np.cos(phi_1dc/2) \
     - d_beta_1 * np.sin(phi_1) * np.sin(phi_1dc/2)
    return _potential

In [ ]:
U_array = [U_1(phi_1, phi_1x, _phi_1dc) for _phi_1dc in np.linspace(1.8, 2, 4)]

In [ ]:
for _U in U_array:
    plt.plot(phi_1, _U)
plt.vlines(0, ymin = 0, ymax = 2.5)
plt.ylim(0, 2.5)

In [ ]:
# By tracking minimum points

In [ ]:
phi_1x = 0
all_minimum_point = []
mirror_min = []
_phi_1dc_array = np.linspace(0, 3, 4)
for _phi_1dc in _phi_1dc_array:
    # U_1(phi_1, phi_1x, _phi_1dc):
    def Fcn(phi_1):
        _potential = 1/2 * (phi_1 - phi_1x)**2 + beta_1 * np.cos(phi_1) * np.cos(_phi_1dc/2) \
         - d_beta_1 * np.sin(phi_1) * np.sin(_phi_1dc/2)
        return _potential
    

    solution_set = [optimize.fmin(Fcn, _g, disp=None) for _g in [-2, 2]]
    # solution_set = [optimize.fsolve(Fcn, _g) for _g in [-2, 2]]
    solution_set = [sol[0] for sol in solution_set]
    mirror_min.append([abs(min(solution_set)), -abs(min(solution_set))])
    # all_minimum_point.append([(sol[0], Fcn(sol[0])) for sol in solution_set])

    all_minimum_point.append(solution_set)

In [ ]:
correct_phi_1x_array = []
corrected_U = []
difference_of_min = []
for _phi_1dc, min_point in zip(_phi_1dc_array, all_minimum_point):
    phi_1_a, phi_1_b = min_point[0], min_point[1]
    correct_phi_1x = 1/2 * (phi_1_a + phi_1_b) - 1/(phi_1_a - phi_1_b) * \
                     (d_beta_1 * (np.sin(phi_1_a) - np.sin(phi_1_b)) * np.sin(_phi_1dc/2) -\
                     beta_1 * np.cos(_phi_1dc/2) * (np.cos(phi_1_a) - np.cos(phi_1_b)))
    
    
    difference = U_1(phi_1_a, correct_phi_1x, _phi_1dc) - U_1(phi_1_b, correct_phi_1x, _phi_1dc) 
    correct_phi_1x_array.append(correct_phi_1x)
    difference_of_min.append(difference)
    corrected_U.append(U_1(phi_1, correct_phi_1x, _phi_1dc))

In [ ]:
color = ["blue" ,"orange", "green", "red"]
for _U, min_point, _c in zip(corrected_U, all_minimum_point, color):
    plt.plot(phi_1, _U, color=_c)
    plt.vlines(min_point[0], ymin = 0, ymax = 2.5, colors=_c)
    plt.vlines(min_point[1], ymin = 0, ymax = 2.5, colors=_c)
plt.ylim(0, 2.5)

In [ ]:
correct_phi_1x_array = []
corrected_U = []
difference_of_min = []
for _phi_1dc, min_point in zip(_phi_1dc_array, mirror_min):
    phi_1_a, phi_1_b = min_point[0], min_point[1]
    correct_phi_1x = 1/2 * (phi_1_a + phi_1_b) - 1/(phi_1_a - phi_1_b) * \
                     (d_beta_1 * (np.sin(phi_1_a) - np.sin(phi_1_b)) * np.sin(_phi_1dc/2) -\
                     beta_1 * np.cos(_phi_1dc/2) * (np.cos(phi_1_a) - np.cos(phi_1_b)))
    
    
    difference = U_1(phi_1_a, correct_phi_1x, _phi_1dc) - U_1(phi_1_b, correct_phi_1x, _phi_1dc) 
    correct_phi_1x_array.append(correct_phi_1x)
    difference_of_min.append(difference)
    corrected_U.append(U_1(phi_1, correct_phi_1x, _phi_1dc))

In [ ]:
color = ["blue" ,"orange", "green", "red"]
for _U, min_point, _c in zip(corrected_U, mirror_min, color):
    plt.plot(phi_1, _U, color=_c)
    plt.vlines(min_point[0], ymin = 0, ymax = 2.5, colors=_c)
    plt.vlines(min_point[1], ymin = 0, ymax = 2.5, colors=_c)
plt.ylim(0, 2.5)

In [ ]:
all_minimum_point